In [1]:
import pandas as pd

| order_number | customer_number |
| ------------ | --------------- |
| 3            | 5               |
| 5            | 1               |
| 6            | 5               |
| 7            | 4               |
| 8            | 6               |
| 9            | 2               |
| 10           | 4               |
| 11           | 16              |
| 12           | 3               |
| 13           | 5               |
| 14           | 3               |
| 15           | 16              |

In [3]:
# make data
df_original = pd.DataFrame({
    "order_number" : [3,5,6,7,8,9, 10, 11, 12,13,14,15],
    "customer_number": [5,1,5,4,6,2,4,16,3,5,3,16]
})
df = df_original.copy()

In [4]:
df.head()

,order_number,customer_number
0,3,5
1,5,1
2,6,5
3,7,4
4,8,6


In [5]:
# group by customer_number, aggrigate order_number 

In [33]:
df_agg = df.groupby("customer_number").agg("count")
df_agg

,order_number
customer_number,
1,1
2,1
3,2
4,2
5,3
6,1
16,2


In [13]:
max_order = df_agg["order_number"].max()
print(max_order)

26


In [31]:
df_agg[df_agg["order_number"]==max_order].head(1).reset_index()[["customer_number"]]

,customer_number
0,3


In [34]:
import pandas as pd

def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    df = orders
    # agg customer order number
    df_agg = df.groupby("customer_number").agg("count")
    # get max order number
    max_order = df_agg["order_number"].max()
    # make result 
    result_df = df_agg[df_agg["order_number"]==max_order].head(1).reset_index()[["customer_number"]]
    return result_df

In [35]:
# SQL part

In [40]:
import pandasql as ps

In [83]:
orders = df_original.copy()

In [76]:
sql_query = """
SELECT customer_number
FROM orders
GROUP BY customer_number 
ORDER BY COUNT(order_number) DESC
LIMIT 1;
"""

In [77]:
ps.sqldf(sql_query, locals())

,customer_number
0,5


In [78]:
sql_query = """
SELECT customer_number
FROM orders
GROUP BY customer_number
HAVING COUNT(order_number) = (SELECT MAX(count_orders) FROM (
  SELECT customer_number, COUNT(*) AS count_orders
  FROM orders
  GROUP BY customer_number
));

"""

In [79]:
ps.sqldf(sql_query, locals())

,customer_number
0,5



| Column Name     | Type     |
| ------------ | --------------- |
| order_number    | int      |
| customer_number | int      |


In [88]:
sql_query = """ 
SELECT MAX(order_count)
FROM (
    SELECT customer_number, COUNT(order_number) as order_count
    FROM orders
    GROUP BY customer_number
)
"""

In [89]:
ps.sqldf(sql_query, locals())

,MAX(order_count)
0,3


,customer_number,order_count
0,5,12


In [100]:
sql_query = """ 
SET @max_order_count = 
SELECT MAX(order_count)
FROM (
    SELECT customer_number, COUNT(order_number) as order_count
    FROM orders
    GROUP BY customer_number
)
;

SELECT @max_order_count;
"""

In [101]:
ps.sqldf(sql_query, locals())

PandaSQLException: (sqlite3.OperationalError) near "SET": syntax error
[SQL:  
SET @max_order_count = 
SELECT MAX(order_count)
FROM (
    SELECT customer_number, COUNT(order_number) as order_count
    FROM orders
    GROUP BY customer_number
)
;

SELECT @max_order_count;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [106]:
sql_query = """ 
WITH order_count_table AS (
    SELECT customer_number, COUNT(order_number) as orders_count
    FROM orders
    GROUP BY customer_number
)
SELECT *
FROM order_count_table
"""
ps.sqldf(sql_query, locals())

,customer_number,orders_count
0,1,1
1,2,1
2,3,2
3,4,2
4,5,3
5,6,1
6,16,2


In [111]:
sql_query = """ 
SELECT MAX(orders_count)
FROM (
    SELECT customer_number, COUNT(order_number) as orders_count
    FROM orders
    GROUP BY customer_number
)

"""
ps.sqldf(sql_query, locals())

,MAX(orders_count)
0,3


In [119]:
sql_query = """ 
WITH max_orders_count AS (
    SELECT MAX(orders_count) as max_order
    FROM (
        SELECT customer_number, COUNT(order_number) as orders_count
        FROM orders
        GROUP BY customer_number
    )
)

SELECT customer_number
FROM orders
GROUP BY customer_number
HAVING COUNT(order_number) = (
    SELECT max_order
    from max_orders_count
)
"""
ps.sqldf(sql_query, locals())

,customer_number
0,5
